In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Substance_Use_Disorder"
cohort = "GSE159676"

# Input paths
in_trait_dir = "../../input/GEO/Substance_Use_Disorder"
in_cohort_dir = "../../input/GEO/Substance_Use_Disorder/GSE159676"

# Output paths
out_data_file = "../../output/preprocess/Substance_Use_Disorder/GSE159676.csv"
out_gene_data_file = "../../output/preprocess/Substance_Use_Disorder/gene_data/GSE159676.csv"
out_clinical_data_file = "../../output/preprocess/Substance_Use_Disorder/clinical_data/GSE159676.csv"
json_path = "../../output/preprocess/Substance_Use_Disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Portal fibroblasts with mesenchymal stem cell features form a reservoir of proliferative myofibroblasts in liver fibrosis"
!Series_summary	"Based on the identification  of a transcriptomic signature, including Slit2,  characterizing portal mesenchymal stem cells  (PMSC)  and derived myofibroblast (MF), we examined  the gene expression profile of in liver tissue derived  from multiple human liver disorders, including  primary sclerosing cholangitis (PSC) (n=12), non-alcoholic steatohepatitis (NASH)  (n=7) and other liver diseases (i.e., primary biliary cholangitis, autoimmune hepatitis, alcoholic liver disease and haemochromatosis) (n=8) and  compared them to healthy controls (tumor free tissue from livers with metastasis from colorectal cancer) (n=5). We found that  SLIT2 was overexpressed in the liver of patients with NASH, PSC and other chronic liver diseases. We also examined the microarray data of the human liver tissue samples for the transcr

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import json
import pandas as pd
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# From the background info, this appears to be a microarray gene expression study
# "The Affymetrix Human Gene 1.0 st array was used" indicates gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Identify keys in sample characteristics

# Trait (Substance Use Disorder) - Row 0 contains condition information
trait_row = 0

# Age - Not available in the sample characteristics
age_row = None

# Gender - Not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value: str) -> int:
    """
    Convert the liver condition/disease to binary trait values (Substance Use Disorder).
    
    In this dataset, the condition is related to liver diseases, not substance use disorder.
    However, alcohol-related liver disease can be considered as substance use disorder.
    """
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: 1 for alcohol-related, 0 for others
    if "alcohol" in value.lower():
        return 1
    else:
        return 0

def convert_age(value: str) -> Optional[float]:
    """
    Convert age values to continuous values.
    Not available in this dataset.
    """
    return None

def convert_gender(value: str) -> Optional[int]:
    """
    Convert gender values to binary (0 for female, 1 for male).
    Not available in this dataset.
    """
    return None

# 3. Save Metadata
# Check if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load clinical data
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    if os.path.exists(clinical_data_path):
        clinical_data = pd.read_csv(clinical_data_path)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview = preview_df(selected_clinical_df)
        print("Clinical data preview:", preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save clinical data to CSV
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 59
Header line: "ID_REF"	"GSM4837490"	"GSM4837491"	"GSM4837492"	"GSM4837493"	"GSM4837494"	"GSM4837495"	"GSM4837496"	"GSM4837497"	"GSM4837498"	"GSM4837499"	"GSM4837500"	"GSM4837501"	"GSM4837502"	"GSM4837503"	"GSM4837504"	"GSM4837505"	"GSM4837506"	"GSM4837507"	"GSM4837508"	"GSM4837509"	"GSM4837510"	"GSM4837511"	"GSM4837512"	"GSM4837513"	"GSM4837514"	"GSM4837515"	"GSM4837516"	"GSM4837517"	"GSM4837518"	"GSM4837519"	"GSM4837520"	"GSM4837521"	"GSM4837522"
First data line: 7896754	6.173769	5.884727	5.867275	5.523547	5.396614	5.352139	7.054756	6.74192	6.56965	7.437846	7.371925	6.912228	6.752769	6.143552	5.795213	5.637336	6.184508	4.983879	6.406087	5.728195	5.290517	6.282499	6.133426	5.255298	5.955746	5.710322	5.67965	5.692214	5.864211	5.47867	5.240821	5.92336	5.656408
Index(['7896754', '7896759', '7896761', '7896779', '7896798', '7896817',
       '7896822', '7896859', '7896863', '7896865', '7896878', '7896882',
       '7896908', '7896917', '7896921', '7896929', '78969

### Step 4: Gene Identifier Review

In [5]:
# Examining the identifier format
# The identifiers in the data appear to be numerical IDs (7896754, 7896759, etc.)
# These are not standard human gene symbols, which typically have alphanumeric forms like "BRCA1", "TP53", etc.
# These appear to be Illumina probe IDs or similar microarray identifiers that need mapping to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE159676
Line 6: !Series_title = Portal fibroblasts with mesenchymal stem cell features form a reservoir of proliferative myofibroblasts in liver fibrosis
Line 7: !Series_geo_accession = GSE159676
Line 8: !Series_status = Public on Oct 21 2020
Line 9: !Series_submission_date = Oct 20 2020
Line 10: !Series_last_update_date = Mar 16 2022
Line 11: !Series_pubmed_id = 35278227
Line 12: !Series_summary = Based on the identification  of a transcriptomic signature, including Slit2,  characterizing portal mesenchymal stem cells  (PMSC)  and derived myofibroblast (MF), we examined  the gene expression profile of in liver tissue derived  from multiple human liver disorders, including  primary sclerosing cholangitis 


Gene annotation preview:
{'ID': [7896736, 7896738, 7896740, 7896742, 7896744], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908', 'NR_024437,XM_006711854,XM_006726377,XR_430662,AK298283,AL137655,BC032332,BC118988,BC122537,BC131690,NM_207366,AK301928,BC071667', 'NM_001005221,NM_001005224,NM_001005277,NM_001005504,BC137547,BC137568'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091', '334129', '367659'], 'RANGE_STOP': ['54936', '63887', '70008', '334296', '368597'], 'total_probes': [7, 31, 24, 6, 36], 'gene_assignment': ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // --- // --- /// ENST000

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify mapping columns
# Looking at the gene_data.index and the gene_annotation dataframe:
# - The gene_data.index contains IDs like '7896754', '7896759', etc.
# - The gene_annotation dataframe has an 'ID' column with similar numeric IDs
# - The gene symbols can be extracted from the 'gene_assignment' column

# 2. Get the gene mapping dataframe
# Extract the ID column and gene_assignment column (which contains gene symbols)
mapping_df = gene_annotation[['ID', 'gene_assignment']]
mapping_df = mapping_df.rename(columns={'gene_assignment': 'Gene'})
mapping_df = mapping_df.astype({'ID': 'str'})

# Preview the mapping dataframe
print("Gene mapping preview:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level data to gene-level data
# Use the apply_gene_mapping function from the library
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Preview the result
print("\nGene expression data after mapping (first 5 genes, first 5 samples):")
print(gene_data.iloc[:5, :5])

# Print some statistics
print(f"\nNumber of genes after mapping: {len(gene_data)}")
print(f"Number of samples: {gene_data.shape[1]}")


Gene mapping preview:
        ID                                               Gene
0  7896736                                                ---
1  7896738  ENST00000328113 // OR4G2P // olfactory recepto...
2  7896740  NM_001004195 // OR4F4 // olfactory receptor, f...
3  7896742  NR_024437 // LOC728323 // uncharacterized LOC7...
4  7896744  NM_001005221 // OR4F29 // olfactory receptor, ...



Gene expression data after mapping (first 5 genes, first 5 samples):
      GSM4837490  GSM4837491  GSM4837492  GSM4837493    GSM4837494
Gene                                                              
A-      9.660575    9.463605    9.569520    9.744110      9.635123
A-52    1.268758    1.284465    1.291198    1.304725      1.274453
A-E     0.381549    0.350529    0.341818    0.359396      0.402815
A-I     3.217762    3.244388    3.280567    3.292618  54173.098551
A-II    2.053555    2.056553    2.044962    2.078925      2.059472

Number of genes after mapping: 93492
Number of samples: 33


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
print(f"Gene data shape before normalization: {gene_data.shape}")

# Normalize gene symbols using NCBI Gene database
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Dataset contains gene expression data but lacks clear trait indicators for {trait} status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note=f"Dataset contains gene expression data for {trait} analysis."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data for {trait}: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Gene data shape before normalization: (93492, 33)
Gene data shape after normalization: (16517, 33)


Normalized gene data saved to ../../output/preprocess/Substance_Use_Disorder/gene_data/GSE159676.csv
Raw clinical data shape: (1, 34)
Clinical features:
                        GSM4837490  GSM4837491  GSM4837492  GSM4837493  \
Substance_Use_Disorder         0.0         0.0         0.0         0.0   

                        GSM4837494  GSM4837495  GSM4837496  GSM4837497  \
Substance_Use_Disorder         0.0         0.0         1.0         1.0   

                        GSM4837498  GSM4837499  ...  GSM4837513  GSM4837514  \
Substance_Use_Disorder         1.0         1.0  ...         0.0         0.0   

                        GSM4837515  GSM4837516  GSM4837517  GSM4837518  \
Substance_Use_Disorder         0.0         0.0         0.0         0.0   

                        GSM4837519  GSM4837520  GSM4837521  GSM4837522  
Substance_Use_Disorder         0.0         1.0         0.0         0.0  

[1 rows x 33 columns]
Clinical features saved to ../../output/preprocess/Substance_Use_Disorde

Data shape after handling missing values: (33, 16518)
For the feature 'Substance_Use_Disorder', the least common label is '1.0' with 8 occurrences. This represents 24.24% of the dataset.
The distribution of the feature 'Substance_Use_Disorder' in this dataset is fine.



Linked data saved to ../../output/preprocess/Substance_Use_Disorder/GSE159676.csv
